In [1]:
import numpy as np
import pandas as pd
import feather
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [2]:
#実際予測時にはその化合物のデータは使えないので、厳密にはGroupKfoldで化合物ごとに分けるべき

In [3]:
features = [
        "Atom",
        "AtomPosition",
        "AtomDistance",
        "CouplingType",
        "AtomEnvironment",
        "AtomNeighbors",
        "BruteForce",
        "ScalarCouplingContributionsOof",
        "DistanceFromClosest",
        "ElectroNegFromClosest",
        "ACSF"]

In [4]:
X_train = pd.concat([feather.read_dataframe("../features/" + feature + "_train.feather") for feature in features], axis=1)
X_test = pd.concat([feather.read_dataframe("../features/" + feature + "_test.feather") for feature in features], axis=1)

In [5]:
X_train.shape, X_test.shape

((4658147, 435), (2505542, 435))

In [6]:
X_train.head()

,atom_0,atom_1,x_0,x_1,y_0,y_1,z_0,z_1,atom_distance,type,...,acsf_115_a1,acsf_116_a1,acsf_117_a1,acsf_118_a1,acsf_119_a1,acsf_120_a1,acsf_121_a1,acsf_122_a1,acsf_123_a1,acsf_124_a1
0,0,0,0.002150,-0.012698,-0.006031,1.085804,0.001976,0.008001,1.544255,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0.002150,1.011731,-0.006031,1.463751,0.001976,0.000277,2.521712,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0.002150,-0.540815,-0.006031,1.447527,0.001976,-0.876644,2.521751,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0.002150,-0.523814,-0.006031,1.437933,0.001976,0.906397,2.521764,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1.011731,-0.012698,1.463751,1.085804,0.000277,0.008001,1.544253,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
contri = feather.read_dataframe("../data/input/scalar_coupling_contributions.feather")

In [8]:
contri.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [9]:
#X_train.drop('PropertyFunctor', axis=1, inplace=True) #always nan
#X_test.drop('PropertyFunctor', axis=1, inplace=True) #always nan

#drop molucules in train with nan descriptors
#replace nan in test with the mean of train
coupling_types = X_train['type']
nan_cols = list(X_train.columns.values[X_train.isnull().any(axis=0)])
coupling_types = coupling_types[~X_train.isnull().any(axis=1)]
contri = contri[~X_train.isnull().any(axis=1)]
X_train = X_train[~X_train.isnull().any(axis=1)]
categorical_cols = list(X_train.columns[X_train.dtypes == object])

for col in nan_cols:
    if col in categorical_cols:
        mode = X_train[col].dropna().mode()
        X_test[col].fillna(mode[0], inplace=True)
    else:
        median = X_train[col].dropna().median()
        X_test[col].fillna(median, inplace=True)

for col in categorical_cols:
    le = LabelEncoder()
    print(f'Starting {col}')
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

Starting a1_is_aromatic
Starting a1_in_ring
Starting a1_in_ring3
Starting a1_in_ring4
Starting a1_in_ring5
Starting a1_in_ring6
Starting a1_in_ring7
Starting a1_in_ring8


In [10]:
oof_train = pd.DataFrame()
oof_test = pd.DataFrame()

for col in ['fc', 'sd', 'pso', 'dso']:
        kf = GroupKFold(n_splits=2)
    
        oof = np.zeros(len(X_train))
        pred = np.zeros(len(X_test))
        
        target = contri[col]
        
        SEED = 42
        NUM_ROUNDS = 10000
        
        params = {
                            "num_leaves": 100,
                            "min_data_in_leaf": 100,
                            "objective": "regression",
                            "max_depth": 10,
                            "learning_rate": 0.2,
                            "boosting_type": "gbdt",
                            "subsample_freq": 1,
                            "subsample": 0.9,
                            "metric": "mae",
                            "reg_alpha": 0.1,
                            "reg_lambda": 0.3, 
                            "colsample_bytree": 0.9
                            }
    
        for train_idx, val_idx in kf.split(X_train, groups=contri['molecule_name']):
                train_data = lgb.Dataset(X_train.iloc[train_idx], label=target.iloc[train_idx], categorical_feature=categorical_cols)
                val_data = lgb.Dataset(X_train.iloc[val_idx], label=target.iloc[val_idx], categorical_feature=categorical_cols)
                clf = lgb.train(params, train_data, NUM_ROUNDS, valid_sets=[train_data, val_data],
                                verbose_eval=1000, early_stopping_rounds=100)
                oof[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)
                pred += clf.predict(X_test, num_iteration=clf.best_iteration) / kf.n_splits
                
        oof_train[col] = oof
        oof_test[col] = pred
        
        print(f'{col} done!')

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.426145	valid_1's l1: 0.485171
[2000]	training's l1: 0.342016	valid_1's l1: 0.435651
[3000]	training's l1: 0.291523	valid_1's l1: 0.411067
[4000]	training's l1: 0.255166	valid_1's l1: 0.395882
[5000]	training's l1: 0.227393	valid_1's l1: 0.386261
[6000]	training's l1: 0.204611	valid_1's l1: 0.37902
[7000]	training's l1: 0.18563	valid_1's l1: 0.373833
[8000]	training's l1: 0.169581	valid_1's l1: 0.36986
[9000]	training's l1: 0.155449	valid_1's l1: 0.366648
[10000]	training's l1: 0.143067	valid_1's l1: 0.364029
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.143067	valid_1's l1: 0.364029
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.426752	valid_1's l1: 0.484036
[2000]	training's l1: 0.34261	valid_1's l1: 0.434072
[3000]	training's l1: 0.291585	valid_1's l1: 0.409373
[4000]	training's l1: 0.255094	valid_1's l1: 0.394236
[5000]	training's 

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0049865	valid_1's l1: 0.00568589
[2000]	training's l1: 0.00398499	valid_1's l1: 0.00505802
[3000]	training's l1: 0.00340253	valid_1's l1: 0.00475063
[4000]	training's l1: 0.00299943	valid_1's l1: 0.00456494
[5000]	training's l1: 0.0026973	valid_1's l1: 0.00444565
[6000]	training's l1: 0.00245413	valid_1's l1: 0.00435865
[7000]	training's l1: 0.00225229	valid_1's l1: 0.00429259
[8000]	training's l1: 0.00208394	valid_1's l1: 0.00424282
[9000]	training's l1: 0.00193928	valid_1's l1: 0.00420169
[10000]	training's l1: 0.00181206	valid_1's l1: 0.00416841
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.00181206	valid_1's l1: 0.00416841
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00499114	valid_1's l1: 0.00564508
[2000]	training's l1: 0.00400039	valid_1's l1: 0.0050277
[3000]	training's l1: 0.00341639	valid_1's l1: 0.00472068
[4000]	training

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0152791	valid_1's l1: 0.0172047
[2000]	training's l1: 0.0121982	valid_1's l1: 0.015258
[3000]	training's l1: 0.0103961	valid_1's l1: 0.0143239
[4000]	training's l1: 0.00912056	valid_1's l1: 0.0137505
[5000]	training's l1: 0.00814403	valid_1's l1: 0.013366
[6000]	training's l1: 0.00735704	valid_1's l1: 0.013083
[7000]	training's l1: 0.00670258	valid_1's l1: 0.0128706
[8000]	training's l1: 0.00614749	valid_1's l1: 0.0127047
[9000]	training's l1: 0.00566908	valid_1's l1: 0.012572
[10000]	training's l1: 0.00525464	valid_1's l1: 0.0124653
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.00525464	valid_1's l1: 0.0124653
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0152344	valid_1's l1: 0.0171407
[2000]	training's l1: 0.0121985	valid_1's l1: 0.0152336
[3000]	training's l1: 0.0103917	valid_1's l1: 0.0142881
[4000]	training's l1: 0.00913608	val

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0101863	valid_1's l1: 0.0117173
[2000]	training's l1: 0.00781152	valid_1's l1: 0.010089
[3000]	training's l1: 0.00655846	valid_1's l1: 0.00939902
[4000]	training's l1: 0.005711	valid_1's l1: 0.00899721
[5000]	training's l1: 0.00507666	valid_1's l1: 0.00873511
[6000]	training's l1: 0.0045753	valid_1's l1: 0.00854905
[7000]	training's l1: 0.00416613	valid_1's l1: 0.00841256
[8000]	training's l1: 0.0038234	valid_1's l1: 0.00830547
[9000]	training's l1: 0.00353234	valid_1's l1: 0.00822364
[10000]	training's l1: 0.00328013	valid_1's l1: 0.00815804
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.00328013	valid_1's l1: 0.00815804
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0102105	valid_1's l1: 0.0116953
[2000]	training's l1: 0.00783643	valid_1's l1: 0.0100782
[3000]	training's l1: 0.00658036	valid_1's l1: 0.00938223
[4000]	training's l1: 0

In [11]:
oof_train.shape, oof_test.shape

((4310225, 4), (2505542, 4))

In [12]:
oof_train.head()

,fc,sd,pso,dso
0,85.365757,0.181236,0.773590,0.446478
1,-9.411008,0.319383,2.638191,-3.386915
2,-9.650752,0.317466,2.631942,-3.385977
3,85.260809,0.180825,0.749124,0.447255
4,-9.608047,0.315361,2.623437,-3.398121


In [13]:
contri.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013
5,dsgdb9nsd_000001,2,3,2JHH,-11.0317,0.352932,2.85856,-3.433950


In [14]:
#X_trainのnanのところにnanを加える（他と形式を合わすため）

In [16]:
X_train = pd.concat([feather.read_dataframe("../features/" + feature + "_train.feather") for feature in features], axis=1)
#X_train.drop('PropertyFunctor', axis=1, inplace=True) #always nan

In [17]:
X_train.head()

,atom_0,atom_1,x_0,x_1,y_0,y_1,z_0,z_1,atom_distance,type,...,acsf_115_a1,acsf_116_a1,acsf_117_a1,acsf_118_a1,acsf_119_a1,acsf_120_a1,acsf_121_a1,acsf_122_a1,acsf_123_a1,acsf_124_a1
0,0,0,0.002150,-0.012698,-0.006031,1.085804,0.001976,0.008001,1.544255,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0.002150,1.011731,-0.006031,1.463751,0.001976,0.000277,2.521712,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0.002150,-0.540815,-0.006031,1.447527,0.001976,-0.876644,2.521751,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0.002150,-0.523814,-0.006031,1.437933,0.001976,0.906397,2.521764,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1.011731,-0.012698,1.463751,1.085804,0.000277,0.008001,1.544253,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
X_train.shape

(4658147, 435)

In [19]:
all_index = X_train.index
nan_index = all_index[X_train.isnull().any(axis=1)]

assert len(all_index) - len(nan_index) == len(oof_train)

In [20]:
feats_train = [[] for _ in range(4)]

count = 0
for idx in all_index:
    if idx in nan_index:
        for j in range(4):
            feats_train[j].append(np.nan)
    else:
        for k in range(4):
            feats_train[k].append(oof_train.iloc[count, k])
        count += 1
            
new_train_oof = pd.DataFrame()
new_train_oof['fc_2'] = feats_train[0]
new_train_oof['sd_2'] = feats_train[1]
new_train_oof['pso_2'] = feats_train[2]
new_train_oof['dso_2'] = feats_train[3]

In [21]:
new_train_oof.shape

(4658147, 4)

In [24]:
new_train_oof.isnull().sum()

fc     347922
sd     347922
pso    347922
dso    347922
dtype: int64

In [34]:
new_train_oof.to_feather('../features/ScalarCouplingContributionsOof2_train.feather')
oof_test.to_feather('../features/ScalarCouplingContributionsOof2_test.feather')